In [6]:
import numpy as np
from sympy import Matrix

class SimplexAlgo:
    def __init__(self, filePath):
        # Read data from CSV file
        arr = np.genfromtxt(filePath, delimiter=',')
        self.z = arr[0, :-1]  # Initial feasible point (shape (n,))
        self.c = arr[1, :-1]  # Cost vector (shape (n,))
        self.b = arr[2:, -1]  # Constraint vector (shape (m,))
        self.A = arr[2:, :-1] # Constraint coefficients matrix (shape (m, n))
        self.n = self.A.shape[1]  # Number of variables
        self.m = self.A.shape[0]  # Number of constraints

        if self.m < self.n:
            raise ValueError("Number of constraints (m) must be at least the number of variables (n).")

        # Original b for degeneracy handling
        self.b_original = self.b.copy()
        self.epsilon = 1e-8  # Small value for degeneracy handling

        # List to store sequence of vertices visited
        self.visited_vertices = []

    def isFeasible(self, z):
        """Check if the solution z is feasible."""
        return np.all(np.matmul(self.A, z) <= self.b + 1e-8)

    def splitRows(self, z):
        """Split A and b into tight and untight constraints based on the current z."""
        Az = np.dot(self.A, z)
        tight = np.isclose(Az, self.b, atol=1e-8)
        untight = Az < self.b - 1e-8

        A1 = self.A[tight]
        b1 = self.b[tight]
        A2 = self.A[untight]
        b2 = self.b[untight]

        # Keep track of indices for Bland's Rule
        self.tight_indices = np.where(tight)[0]
        self.untight_indices = np.where(untight)[0]

        return A1, b1, A2, b2

    def isVertex(self, z):
        """Check if the current solution z is a vertex."""
        A1, _, _, _ = self.splitRows(z)
        rank = np.linalg.matrix_rank(A1)
        return rank == self.n

    def moveTowardsVertex(self, z):
        """Move the feasible point towards a vertex."""
        while not self.isVertex(z):
            A1, _, A2, b2 = self.splitRows(z)

            if A1.size == 0:
                # No tight constraints, move towards the nearest constraint
                slack = self.b - np.dot(self.A, z)
                min_slack_index = np.argmin(slack)
                direction = self.A[min_slack_index]
            else:
                # Compute null space of A1
                nullspace = np.array(Matrix(A1).nullspace())
                if nullspace.size == 0:
                    break  # At a vertex

                direction = nullspace[0].astype(np.float64).flatten()

            direction = direction / np.linalg.norm(direction)
            temp = np.dot(A2, direction)
            temp = np.where(np.abs(temp) < 1e-16, 1e-16, temp)
            alpha_values = (b2 - np.dot(A2, z)) / temp

            # Apply Bland's Rule for choosing alpha
            positive_alphas = alpha_values > 1e-8
            if not np.any(positive_alphas):
                raise ValueError("Cannot find a positive alpha to move towards a vertex.")

            alpha_candidates = alpha_values.copy()
            alpha_candidates[~positive_alphas] = np.inf
            min_alpha = np.min(alpha_candidates)
            min_alpha_indices = np.where(np.isclose(alpha_candidates, min_alpha, atol=1e-8))[0]

            # Choose the constraint with the smallest index (Bland's Rule)
            leaving_constraint_index = self.untight_indices[min_alpha_indices].min()
            alpha = alpha_candidates[min_alpha_indices[np.argmin(self.untight_indices[min_alpha_indices])]]

            z = z + alpha * direction

            # Handle degeneracy if necessary
            if self.isDegenerate(z):
                self.removeDegeneracy()
                z = self.z  # Restart with perturbed b
                continue

        return z

    def isDegenerate(self, z):
        """Check if the current vertex is degenerate."""
        A1, _, _, _ = self.splitRows(z)
        return A1.shape[0] > self.n

    def removeDegeneracy(self):
        """Remove degeneracy by perturbing b slightly."""
        self.epsilon *= 10
        perturbation = self.epsilon ** np.arange(1, self.m + 1)
        self.b = self.b_original + perturbation

    def isOptimal(self, z):
        """Check if the current vertex is optimal."""
        A1, _, _, _ = self.splitRows(z)
        if A1.shape[0] != self.n:
            return False

        try:
            beta = np.linalg.solve(A1.T, self.c)
        except np.linalg.LinAlgError:
            return False

        return np.all(beta >= -1e-8)

    def optimize(self, z):
        """Optimize the cost function by moving to the next vertex."""
        while True:
            A1, _, A2, b2 = self.splitRows(z)

            if A1.shape[0] != self.n:
                raise ValueError("Active constraint matrix A1 must be square.")

            try:
                A1_inv_T = np.linalg.inv(A1).T
            except np.linalg.LinAlgError:
                raise ValueError("Active constraint matrix A1 is singular.")

            beta = np.dot(A1_inv_T, self.c)
            if np.all(beta <= 1e-8):
                # Optimal solution found
                break

            # Apply Bland's Rule for selecting entering variable
            improving_indices = np.where(beta > 1e-8)[0]
            if improving_indices.size == 0:
                break  # No improving direction found

            # Choose variable with the smallest index
            entering_variable_index = self.tight_indices[improving_indices].min()
            idx = np.where(self.tight_indices == entering_variable_index)[0][0]
            u = A1_inv_T[:, idx]

            temp = np.dot(A2, u)
            temp = np.where(np.abs(temp) < 1e-16, 1e-16, temp)
            alpha_values = (b2 - np.dot(A2, z)) / temp

            # Apply Bland's Rule for selecting leaving variable
            positive_alphas = alpha_values > 1e-8
            if not np.any(positive_alphas):
                raise ValueError("Problem is unbounded.")

            alpha_candidates = alpha_values.copy()
            alpha_candidates[~positive_alphas] = np.inf
            min_alpha = np.min(alpha_candidates)
            min_alpha_indices = np.where(np.isclose(alpha_candidates, min_alpha, atol=1e-8))[0]

            # Choose the constraint with the smallest index (Bland's Rule)
            leaving_constraint_index = self.untight_indices[min_alpha_indices].min()
            alpha = alpha_candidates[min_alpha_indices[np.argmin(self.untight_indices[min_alpha_indices])]]

            z = z + alpha * u

            # Handle degeneracy if necessary
            if self.isDegenerate(z):
                self.removeDegeneracy()
                z = self.z  # Restart with perturbed b
                continue

            if not self.isFeasible(z):
                raise ValueError("Solution became infeasible during iteration.")

            self.visited_vertices.append((z.copy(), np.dot(self.c, z).item()))

        return z

    def find_optimal_vertex(self):
        """Find the optimal vertex for the objective function and return the sequence of vertices."""
        z = self.z.copy()
        if not self.isFeasible(z):
            raise ValueError("Initial point z is not feasible.")

        # Move towards a vertex
        z = self.moveTowardsVertex(z)
        self.visited_vertices.append((z.copy(), np.dot(self.c, z).item()))

        # Optimize the objective function
        z = self.optimize(z)
        self.z = z  # Update the current point

        return self.visited_vertices


In [7]:
# Assuming the class SimplexAlgo is already defined as above

# Create an instance of the SimplexAlgo class with the path to your CSV file
simplex = SimplexAlgo(r'C:\Users\gupta\OneDrive - IIT Hyderabad\IITH\Assignment\LO\My Code\3.Dataset.csv')

# Find the optimal vertex
vertices = simplex.find_optimal_vertex()

# Print the sequence of vertices visited and their objective function values
for idx, (vertex, cost) in enumerate(vertices):
    print(f"Vertex {idx + 1}: {vertex}, Objective Value: {cost}")


KeyboardInterrupt: 